### Characterizing the crackle events

In [3]:
import sys

sys.path.insert(1, "/home/vinicius/storage1/projects/GrayData-Analysis")

In [4]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
import seaborn as sns
import xarray as xr
from frites.stats import confidence_interval
from loader import loader
from tqdm import tqdm
from utils import *

from config import get_dates, return_delay_split
from GDa.flatmap.flatmap import flatmap

##### Setting configurations

In [5]:
_ROOT = os.path.expanduser("~/funcog/gda/")

In [6]:
metric = "coh"
monkey = "lucy"
ds = 1

In [7]:
early_cue, early_delay = return_delay_split(monkey, delay_type=ds)

In [8]:
sessions = get_dates(monkey)